In [1]:
!pip install -q torch torchvision torchaudio torch-geometric
!pip install -q faiss-cpu sentence-transformers transformers accelerate
!pip install -q sastrawi networkx matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [2]:
import pandas as pd

# Load data kredit
df_credit = pd.read_csv("https://raw.githubusercontent.com/Muhammad-Ikhwan-Fathulloh/Risk-Intelligence-Engine/main/Dataset/credit_risk_dataset.csv")

# Gabungkan fitur menjadi format text
texts = []
for _, row in df_credit.iterrows():
    teks = (
        f"Umur: {row['person_age']}, "
        f"Penghasilan: {row['person_income']}, "
        f"Kepemilikan rumah: {row['person_home_ownership']}, "
        f"Lama kerja: {row['person_emp_length']}, "
        f"Tujuan pinjaman: {row['loan_intent']}, "
        f"Nilai pinjaman: {row['loan_amnt']}, "
        f"Grade: {row['loan_grade']}, "
        f"Suku bunga: {row['loan_int_rate']}, "
        f"Status pinjaman: {row['loan_status']}, "
        f"Rasio pinjaman terhadap penghasilan: {row['loan_percent_income']}, "
        f"Default sebelumnya: {row['cb_person_default_on_file']}, "
        f"Panjang histori kredit: {row['cb_person_cred_hist_length']}"
    )
    texts.append(teks)

In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = embedding_model.encode(texts, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "GoToCompany/llama3-8b-cpt-sahabatai-v1-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

In [5]:
def query_rag(user_query, top_k=3):
    # Encode query
    query_vec = embedding_model.encode([user_query], convert_to_numpy=True)
    D, I = index.search(query_vec, top_k)

    # Ambil konteks
    contexts = [texts[i] for i in I[0]]
    context_prompt = "\n".join([f"{i+1}. {ctx}" for i, ctx in enumerate(contexts)])

    # Prompt untuk LLM
    prompt = (
        f"Berikut adalah beberapa data peminjam:\n{context_prompt}\n\n"
        f"Pertanyaan: {user_query}\n"
        f"Jawaban:"
    )

    # Generate jawaban
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
question = "Apa risiko dari peminjam muda dengan penghasilan kecil dan grade C?"
answer = query_rag(question)
print("Pertanyaan:", question)
print("Jawaban:\n", answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Pertanyaan: Apa risiko dari peminjam muda dengan penghasilan kecil dan grade C?
Jawaban:
 Berikut adalah beberapa data peminjam:
1. Umur: 23, Penghasilan: 77927, Kepemilikan rumah: MORTGAGE, Lama kerja: 7.0, Tujuan pinjaman: EDUCATION, Nilai pinjaman: 15000, Grade: C, Suku bunga: 12.99, Status pinjaman: 0, Rasio pinjaman terhadap penghasilan: 0.19, Default sebelumnya: Y, Panjang histori kredit: 4
2. Umur: 24, Penghasilan: 112500, Kepemilikan rumah: MORTGAGE, Lama kerja: 3.0, Tujuan pinjaman: EDUCATION, Nilai pinjaman: 8600, Grade: C, Suku bunga: 13.11, Status pinjaman: 0, Rasio pinjaman terhadap penghasilan: 0.08, Default sebelumnya: Y, Panjang histori kredit: 3
3. Umur: 25, Penghasilan: 70224, Kepemilikan rumah: MORTGAGE, Lama kerja: 1.0, Tujuan pinjaman: EDUCATION, Nilai pinjaman: 21000, Grade: C, Suku bunga: 13.22, Status pinjaman: 0, Rasio pinjaman terhadap penghasilan: 0.3, Default sebelumnya: N, Panjang histori kredit: 4

Pertanyaan: Apa risiko dari peminjam muda dengan penghasil